# Reinforcement Learning

O aprendizado por reforço ajuda a desenvolver a melhor forma para executar o trabalho por tentativa e erro, conforme é explorado se calcula a melhor opção que o agente possui para realizar sua função, que neste caso, o taxi tomar a melhor rota.

In [ ]:
import gym
import random

random.seed(1234)

streets = gym.make("Taxi-v3", render_mode='ansi').env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4
streets.reset()
print("\n" + streets.render()) ##printar e renderizar nossas ruas, essa elaboradas no gym.make


-  R, G, B, e Y são os locais para pegar ou deixar passageiros.
-  O azul onde é preciso passar para pegar o passageiro
-  A magenta representa onde o passageiro quer chegar
-  As linhas retas representam as ruas que ele não pode atravessar
-  O retângulo amarelo representa o taxi, quando mantém a cor ele está vazio e verde quando possui um passageiro

-  Lugares acessáveis representam um 5x5 = 25 lugares 
-  Existem 4 possibilidades de destino
- E contando a localização do passageiro se tem 5

Nesse caso, 25 x 4 x 5 = 500 possibilidades 

6 possibilidades de ação:

-  Se movimentar para sul, leste, norte ou oeste
-  Pegar o passageiro
-  Deixá-lo

O Q-Learning dará as seguintes recompensas e penalidades de cada forma:

Deixar o passageiro de forma correta concede +20 pontos
Cada unidade de tempo enquanto dirige com um passageiro gera uma penalidade de -1 ponto
Pegar ou deixar em um local ilegal gera uma penalidade de -10 pontos

Vamos definir um estado inicial, com o taxi na loc (2, 3), com o passageiro a ser pego na 2 e ser deixado na 0

In [ ]:
initial_state = streets.encode(2, 3, 2, 0) ##aqui o código explicado acima

streets.s = initial_state ##variavel para guardar os dados

streets.render() #renderizar 

In [ ]:
streets.P[initial_state] ##printando o estado inicial definido anteriormente e verificando as locs

Mover para o Sul, Norte, Leste ou Oeste, pegar passageiro ou desembarcar. Os quatro valores em cada linha representam a probabilidade atribuída a essa ação, o maior foco é entregar o passageiro ao seu destino

Por exemplo, mover-se para o Norte a partir deste estado nos colocaria no estado número 368, acarretaria uma penalidade de -1 por ocupar tempo e não resultar em um desembarque correto.

Primeiro, treinaremos o modelo. Em um nível geral, faremos o treinamento ao longo de 10mil corridas de táxi. Em cada corrida, avançaremos no tempo, com uma chance de 10% em cada passo de tomar uma ação aleatória em vez de usar os valores Q aprendidos para orientar o agente.

In [ ]:
import numpy as np ##import biblioteca para lidar com os numeros

q_table = np.zeros([streets.observation_space.n, streets.action_space.n]) ##criar tabela

learning_rate = 0.1 ##valor atribuido à "importancia" de cada teste e quanto isso soma pro  aprendizado
discount_factor = 0.6 
exploration = 0.1 ##porcentagem de exploracao sendo 10%
epochs = 10000 ##numero de testes para contabilizar o Qlearning

for taxi_run in range(epochs): ##iteração pros testes
    state, _ = streets.reset() #para pegar apenas o estado inicial, sem isso daria erro pela versao
    done = False
    truncated = False ##para calcular corretamente quando o agente acabou de fato seu aprendizado
    
    while not (done or truncated):
        random_value = random.uniform(0, 1) ##variança de 0 a 1 para uso aleatorio ou qlearning
        if (random_value < exploration):
            action = streets.action_space.sample()  ##random
        else:
            action = np.argmax(q_table[state]) ##qlearning
            
        next_state, reward, done, truncated, info = streets.step(action) ##estado, recompensa, se finalizou ou parou e info adicionais
        
        prev_q = q_table[state, action] ##atualizacao de valores e substituição dos antigos valores nos códigos abaixo
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state, action] = new_q
        
        state = next_state
        
        

In [ ]:
q_table[initial_state]

In [ ]:
from time import sleep

for tripnum in range(1, 11): ##10 viagens
    state, _ = streets.reset() ##zerar o ambiente
    done = False
    truncated = False
    trip_length = 0 ##duração
    
    while not done and truncated and trip_length < 25: ##para cada viagem não ter passos demais e o teste perder o sentido 
        
        action = np.argmax(q_table[state]) ##buscar ação
        next_state, reward, done, truncated, info = streets.step(action) 
        
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(streets.render()) ##printar a execução do taxi
        sleep(.5) ##tempo de espera
        state, _ = next_state
        trip_length += 1 ##incrementador
        
    sleep(2)
    

## Your Challenge

Modify the block above to keep track of the total time steps, and use that as a metric as to how good our Q-learning system is. You might want to increase the number of simulated trips, and remove the sleep() calls to allow you to run over more samples.

Now, try experimenting with the hyperparameters. How low can the number of epochs go before our model starts to suffer? Can you come up with better learning rates, discount factors, or exploration factors to make the training more efficient? The exploration vs. exploitation rate in particular is interesting to experiment with.

In [ ]:
import gym
import random
import numpy as np
from time import sleep


random.seed(1234)

streets = gym.make("Taxi-v3", render_mode='ansi').env
streets.reset()
print("\n" + streets.render())

In [ ]:
q_table = np.zeros([streets.observation_space.n, streets.action_space.n])
learning_rate = 0.1 
discount_factor = 0.6  
exploration = 0.1 
epochs = 10000  

In [ ]:
for taxi_run in range(epochs):
    state, _ = streets.reset()
    done = False
    truncated = False

    while not (done or truncated):
        random_value = random.uniform(0, 1)
        if random_value < exploration:
            action = streets.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, truncated, info = streets.step(action)

        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state, action] = new_q

        state = next_state


In [ ]:
total_time_steps = 0 #variavel para cada passo do taxi
num_trips = 100  #numero de viagens 

for tripnum in range(1, num_trips + 1):
    state, _ = streets.reset()
    done = False
    truncated = False
    trip_length = 0

    while not (done or truncated) and trip_length < 25:
        action = np.argmax(q_table[state])
        next_state, reward, done, truncated, info = streets.step(action)
        state = next_state
        trip_length += 1

    total_time_steps += trip_length  #acumula cada passo do taxi

print(f"Total time steps in {num_trips} travel: {total_time_steps}") #printando cada passo por viagem

Vamos criar hiperparâmetros aleatórios para fazer várias verificações diferentes nas viagens, todos esses parâmetros são explicados anteriormente do porquê existirem e sua importânncia.

In [ ]:
hyperparameters = [
    {'learning_rate': 0.1, 'discount_factor': 0.6, 'exploration': 0.1, 'epochs': 10000},
    {'learning_rate': 0.7, 'discount_factor': 0.2, 'exploration': 0.3, 'epochs': 7000},
    #{'learning_rate': 0.5, 'discount_factor': 0.9, 'exploration': 0.2, 'epochs': 10000},
    ]

In [ ]:
#iteração dos parametros sob os listados acima
for params in hyperparameters:
    learning_rate = params['learning_rate']
    discount_factor = params['discount_factor']
    exploration = params['exploration']
    epochs = params['epochs']

    q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

    for taxi_run in range(epochs):
        state, _ = streets.reset()
        done = False
        truncated = False

        while not (done or truncated):
            random_value = random.uniform(0, 1)
            if random_value < exploration:
                action = streets.action_space.sample()
            else:
                action = np.argmax(q_table[state])

            next_state, reward, done, truncated, info = streets.step(action)

            prev_q = q_table[state, action]
            next_max_q = np.max(q_table[next_state])
            new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
            q_table[state, action] = new_q

            state = next_state

    total_time_steps = 0

    for tripnum in range(1, num_trips + 1):
        state, _ = streets.reset()
        done = False
        truncated = False
        trip_length = 0

        while not (done or truncated) and trip_length < 25:
            action = np.argmax(q_table[state])
            next_state, reward, done, truncated, info = streets.step(action)
            #print(streets.render())  #mostrar o taxi sendo testado
            trip_length += 1
            
            state = next_state

            #sleep(0.1)

        total_time_steps += trip_length #iterar os pasos com a distância
        

    print(f"Hyperparameters: {params} | Total time steps: {total_time_steps}") #printar os hiperparametros setados junto dos passos que foram necessários

Ao final da atividade e com inclusão de várias novas etapas é possível retirar algumas conclusões: 
Utilizar os hiperparametros é muito interessantes para testar as diferentes formas que um programa pode aprender com mudanças no seu modelo de teste, pode-se ver o taxi em movimento e como ele se comportará em cada mapa diferente baseado nos locais de pickoff e dropoff, ou seja, onde pegará e deixará o passageiro, podemos testar o número de viagens, taxa de aprendizado, dados coletados, os passos a cada viagem e cada teste. O Q-learning é muito interessante para o agente criar e otimizar seu trabalho, neste modelo, buscando as melhores rotas.